In [ ]:
# @authors: Raj Vardhan

<h1> Explanation based detection of adversarial samples for Twitter</h1>
<br>
<p>
This notebook trains a classifier on the explanations of benign (class 0) and adversarial (class 1) samples.
The explanations of a sample are in the form of a binary feature vector (same dimension as sample) where top k indices with highest gradient are 1 and rest are 0

In [1]:
import tensorflow as tf 
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from cleverhans.attacks import SaliencyMapMethod,FastGradientMethod,CarliniWagnerL2,DeepFool
from cleverhans.utils import other_classes, set_log_level
from cleverhans.utils import pair_visual, grid_visual, AccuracyReport
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import model_train, model_eval, model_argmax
from cleverhans.utils_keras import KerasModelWrapper, cnn_model
from cleverhans.model import *

import pandas
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dense, Dropout, Activation
import pickle
import operator
from keras.models import Sequential
from graphviz import Digraph

### Load data and model
import os
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf
import pickle
import datetime
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
seed = 7
np.random.seed(seed)

In [4]:
directory = "./twitter_data_new"

In [5]:
#Preprocessing
gradB = np.load(directory+'/grads_sorted_ind_benign.npy')
gradAdv = np.load(directory+'/grads_sorted_ind_adv.npy')

#initialize explanation matrices
eB = np.zeros(gradB.shape)
eAdv = np.zeros(gradAdv.shape)

#how many top indices should be considered?
thresh = 8

#set top indices to 1 in the overall feature vector (same dimension as input samples)
for ind in range(gradB.shape[0]):
    eB[ind][ gradB[ind][0:thresh] ] = 1

for ind in range(gradAdv.shape[0]):
    eAdv[ind][ gradAdv[ind][0:thresh] ] = 1
    
no_of_col = eB.shape[1]
print('no. of columns is {}'.format(no_of_col))


# Prepare X and Y for training and testing
X_orig = np.vstack((eB, eAdv))

yB = np.zeros((eB.shape[0], 1))
yAdv = np.ones((eAdv.shape[0], 1))

Y_orig = np.vstack((yB, yAdv))

#shuffle the order
indices = np.random.permutation(X_orig.shape[0])
X = X_orig[indices]
Y = Y_orig[indices]

no. of columns is 28


In [6]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [7]:
print('x_train shape : {}'.format(X_train.shape))
print('y_train shape : {}'.format(y_train.shape))
print('x_test shape : {}'.format(X_test.shape))
print('y_test shape : {}'.format(y_test.shape))

x_train shape : (5554, 28)
y_train shape : (5554, 1)
x_test shape : (2736, 28)
y_test shape : (2736, 1)


In [ ]:
'''
# All indices where the class is Malicious
ind=np.where(Y==1)

xM=X[ind,:]

# xM is malware samples
pickle.dump(xM,open(r"xmW_twitter.pickle","wb"))
'''

In [8]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

num_hidden_1 = 20
num_hidden_2 = 20

# build DNNs
DNNmodel = Sequential()
DNNmodel.add(Dense(num_hidden_1, input_dim = no_of_col, kernel_initializer="uniform",activation="relu"))
DNNmodel.add(Dropout(0.5))
DNNmodel.add(Dense(num_hidden_1, kernel_initializer="uniform", activation="relu"))
DNNmodel.add(Dropout(0.5))
DNNmodel.add(Dense(2))
DNNmodel.add(Activation("softmax"))

/Users/raj/anaconda2/envs/xai-env27/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/raj/anaconda2/envs/xai-env27/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
#compile Model
DNNmodel.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# 3-fold validations
history=DNNmodel.fit(X_train, dummy_y,validation_split=0.33,
          epochs=100, batch_size=256)
#DNNmodel.summary()

#evaluation
score = DNNmodel.evaluate(X_train, dummy_y, batch_size=256)
print("Test Accuracy",score)
print("\n%s: %.2f%%" % (DNNmodel.metrics_names[1], score[1]*100))

#store the model weights and bias
#W=DNNmodel.get_weights()
#pickle.dump(W,open(r"W_exp_drebin.pickle","wb"))

Train on 3721 samples, validate on 1833 samples
Epoch 1/100
3721/3721 [==============================] - 1s 144us/step - loss: 0.6771 - acc: 0.6719 - val_loss: 0.6431 - val_acc: 0.7965
Epoch 2/100
3721/3721 [==============================] - 0s 12us/step - loss: 0.6124 - acc: 0.7987 - val_loss: 0.5623 - val_acc: 0.8161
Epoch 3/100
3721/3721 [==============================] - 0s 15us/step - loss: 0.5387 - acc: 0.8269 - val_loss: 0.4878 - val_acc: 0.8292
Epoch 4/100
3721/3721 [==============================] - 0s 12us/step - loss: 0.4769 - acc: 0.8449 - val_loss: 0.4271 - val_acc: 0.8554
Epoch 5/100
3721/3721 [==============================] - 0s 12us/step - loss: 0.4314 - acc: 0.8441 - val_loss: 0.3845 - val_acc: 0.8571
Epoch 6/100
3721/3721 [==============================] - 0s 14us/step - loss: 0.3957 - acc: 0.8624 - val_loss: 0.3574 - val_acc: 0.8669
Epoch 7/100
3721/3721 [==============================] - 0s 13us/step - loss: 0.3768 - acc: 0.8707 - val_loss: 0.3415 - val_acc: 0.8696

3721/3721 [==============================] - 0s 13us/step - loss: 0.2756 - acc: 0.9046 - val_loss: 0.2584 - val_acc: 0.8980
Epoch 61/100
3721/3721 [==============================] - 0s 13us/step - loss: 0.2672 - acc: 0.9051 - val_loss: 0.2577 - val_acc: 0.8985
Epoch 62/100
3721/3721 [==============================] - 0s 12us/step - loss: 0.2638 - acc: 0.9043 - val_loss: 0.2571 - val_acc: 0.8974
Epoch 63/100
3721/3721 [==============================] - 0s 12us/step - loss: 0.2691 - acc: 0.9102 - val_loss: 0.2567 - val_acc: 0.9013
Epoch 64/100
3721/3721 [==============================] - 0s 12us/step - loss: 0.2696 - acc: 0.9059 - val_loss: 0.2569 - val_acc: 0.9018
Epoch 65/100
3721/3721 [==============================] - 0s 17us/step - loss: 0.2698 - acc: 0.9086 - val_loss: 0.2559 - val_acc: 0.9002
Epoch 66/100
3721/3721 [==============================] - 0s 23us/step - loss: 0.2665 - acc: 0.9078 - val_loss: 0.2549 - val_acc: 0.9013
Epoch 67/100
3721/3721 [==============================

In [ ]:
#thresh=15 acc: 88.67%
#thres=12  acc: 90.05%
#thresh=8  acc: 94.75%

In [10]:
y_pred = DNNmodel.predict(X_test)
y_pred_am = np.argmax(y_pred, axis=1)

tp=0
fp=0
tn=0
fn=0

for i in range(len(y_pred_am)):
    if y_test[i][0] == 1:
        if y_pred_am[i] == y_test[i]:
            tp = tp+1
        else:
            fn = fn+1
    
    elif y_test[i][0] == 0:
        if y_pred_am[i] == y_test[i]:
            tn = tn+1
        else:
            fp = fp+1

In [11]:
print('false positives: {} out of {} possible negatives (benign)'.format(fp, fp+tn))
print('false negatives: {} out of {} possible positives (adversarial)'.format(fn, fn+tp))
print('precision: {}%'.format(tp*100.0/(tp+fp)))
print('recall: {}%'.format(tp*100.0/(tp+fn)))

false positives: 104 out of 1258 possible negatives (benign)
false negatives: 155 out of 1478 possible positives (adversarial)
precision: 92.71198318149965%
recall: 89.5128552097429%
